In [1]:
%load_ext autoreload
%autoreload 2

In this example we will create alkyl chains of various lengths, run quantum chemical analysis on each and then replicate them into a simulation cell for an MD simulation

In [2]:
from __future__ import division, unicode_literals

In [3]:
import os 
from pprint import pprint

In [4]:
import streamm

Now let's create project and resource to keep track of our work

In [5]:
alkyl_example = streamm.Project('alkyl_example')
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [6]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [7]:
print res_local.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Create the local directories that will store our files 

In [8]:
res_local.make_dir()

Tell the project about our directories 

In [9]:
alkyl_example.set_resource(res_local)

Read in the methane.xyz file created in the structure.ipynb example 

In [10]:
methane = streamm.Buildingblock('methane')

In [11]:
methane.read_xyz()

Create the neighbor list and the bonded interactions

In [12]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

In [13]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [14]:
print methane.n_particles

5


In [15]:
print methane.print_properties()

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


Set the paramkeys so we can identify force field paramters later on

In [16]:
for pkey,p in methane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [17]:
for pk,p in methane.particles.iteritems():
    p.residue = 1
    p.resname = 'METH'

Set some rsites to be able to join molecules together 

In [18]:
methane.particles[1].rsite = 'RH'
methane.particles[2].rsite = 'RH'

In [19]:
methane.find_rsites()

In [20]:
print methane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[2] H (H) index:2 n_bonds:1] 



Read in ethane.xyz from the buildinblock.ipynb example 

In [21]:
ethane = streamm.Buildingblock('ethane')

In [22]:
ethane.read_xyz()

In [23]:
ethane.bonded_nblist = ethane.guess_nblist(0,radii_buffer=1.25)

In [24]:
ethane.bonded_bonds()
ethane.bonded_angles()
ethane.bonded_dih()

In [25]:
print ethane.n_particles

8


In [26]:
print ethane.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


In [27]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [28]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

In [29]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

In [30]:
ethane.find_rsites()

In [31]:
print ethane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[5] H (H) index:5 n_bonds:1] 



In [32]:
import copy

In [33]:
octane = copy.deepcopy(ethane)

In [34]:
from streamm.structures.buildingblock import attach

In [35]:
for i in range(3):
    octane = attach(octane,ethane,'RH',1,'RH',0)

In [36]:
octane.tag = 'octane'

In [37]:
for pk,p in octane.particles.iteritems():
    p.residue = 2
    p.resname = "OCT"
 

In [38]:
octane.write_xyz()

In [39]:
print octane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[23] H (H) index:23 n_bonds:1] 



In [40]:
print octane.particles[14].symbol

H


In [41]:
octane.particles[14].rsite = 'R2'

In [42]:
octane.find_rsites()

In [43]:
ethyl_octane = attach(octane,ethane,'R2',0,'RH',0)

In [44]:
ethyl_octane.tag = 'ethyl_octane'

In [45]:
ethyl_octane.write_xyz()

In [46]:
for pk,p in ethyl_octane.particles.iteritems():
    print p.residue,p.resname
    #p.residue = 1
    #p.resname = 'ETHHEX'

2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
2 OCT
1 ETH
1 ETH
1 ETH
1 ETH
1 ETH
1 ETH
1 ETH


In [47]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [48]:
print oplsaa


    Parameters 
      LJ parameters 2 
      Bond parameters 3 
      Angle parameters 2 
      Dihedral parameters 1 
      Imporper Dihedral parameters 0 



In [49]:
nwchem_i = streamm.NWChem('nw_ethane_HF')

In [50]:
alkyl_example.add_calc(nwchem_i)

In [51]:
nwchem_i.strucC = ethane

In [52]:
nwchem_i.set_resource(res_local)

In [53]:
nwchem_i.make_dir()

In [54]:
os.chdir(nwchem_i.dir['scratch'])

In [55]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [56]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [57]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

In [58]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_i.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [59]:
pprint(nwchem_i.properties)

{u'allocation': u'',
 u'basis': u'6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/nw_ethane_HF/',
 u'spin_mult': 1,
 u'task': u'SCF ',
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


In [60]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

In [61]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_i.tag))

In [62]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

In [63]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.dump_json()

In [66]:
os.chdir(nwchem_i.dir['scratch'])

In [67]:
nwchem_i.run()

In [68]:
alkyl_example.check()

Calculation nw_ethane_HF has status running


In [69]:
nwchem_i.analysis()

In [70]:
nwchem_i.store()

In [71]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.dump_json()

In [72]:
gaussian_i = streamm.Gaussian('gaus_ethane_HF')

In [73]:
alkyl_example.add_calc(gaussian_i)

In [74]:
gaussian_i.strucC = ethane

In [75]:
gaussian_i.set_resource(res_local)

In [76]:
gaussian_i.make_dir()

In [77]:
os.chdir(gaussian_i.dir['scratch'])

In [78]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [79]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [80]:
gaussian_i.load_str('templates','com')        
gaussian_i.load_str('templates','run')

In [81]:
gaussian_i.properties['commands'] = 'HF/3-21G SP'
gaussian_i.properties['method'] = 'UHF'
gaussian_i.properties['charge'] = 0
gaussian_i.properties['spin_mult'] = 1
gaussian_i.properties['coord'] = gaussian_i.strucC.write_coord()

In [82]:
pprint(gaussian_i.properties)

{u'allocation': u'',
 u'charge': 0,
 u'commands': u'HF/3-21G SP',
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Normal termination of Gaussian',
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/gaus_ethane_HF/',
 u'spin_mult': 1,
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


In [83]:
gaussian_i.replacewrite_prop('com','input','com','%s.com'%(gaussian_i.tag))

In [84]:
gaussian_i.properties['input_com'] = gaussian_i.files['input']['com']
gaussian_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaussian_i.tag))

In [85]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaussian_i.tag)
gaussian_i.add_file(file_type,file_key,file_name)

In [86]:
os.chdir(gaussian_i.dir['home'])
alkyl_example.dump_json()

In [87]:
os.chdir(gaussian_i.dir['scratch'])

In [88]:
gaussian_i.run()

In [89]:
alkyl_example.check()

Calculation nw_ethane_HF has status running
Calculation gaus_ethane_HF has status running


In [90]:
os.chdir(alkyl_example.dir['home'])
alkyl_example.dump_json()

In [91]:
lmp_alkyl = streamm.LAMMPS('lmp_alkyl')

In [92]:
alkyl_example.add_calc(lmp_alkyl)

In [93]:
lmp_alkyl.set_resource(res_local)

In [94]:
lmp_alkyl.make_dir()

In [113]:
print lmp_alkyl.dir['scratch']

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_alkyl/


In [110]:
os.chdir(lmp_alkyl.dir['scratch'])

In [95]:
lmp_alkyl.paramC = oplsaa

In [96]:
import streamm

In [102]:
lmp_alkyl.strucC =  streamm.Buildingblock(matrix=[50.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,50.0])

In [103]:
lmp_alkyl.strucC.lat.pbcs = [True,True,True]

In [104]:
seed = 92734
lmp_alkyl.strucC = streamm.add_struc(lmp_alkyl.strucC,ethyl_octane,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


In [105]:
print lmp_alkyl.strucC.lat

50.000000 0.000000 0.000000
0.000000 50.000000 0.000000
0.000000 0.000000 50.000000


In [106]:
print lmp_alkyl.strucC.n_molecules()

9


In [107]:
print ethyl_octane.tag

ethyl_octane


In [108]:
lmp_alkyl.strucC.tag = ethyl_octane.tag + '_x10'

In [112]:
lmp_alkyl.strucC.write_xyz()

In [114]:
seed = 283674
lmp_alkyl.strucC = streamm.add_struc(lmp_alkyl.strucC,ethane,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


In [115]:
print lmp_alkyl.strucC.n_molecules()

19


In [116]:
lmp_alkyl.strucC.tag += '_ethane_x10'

In [117]:
lmp_alkyl.strucC = streamm.add_struc_grid(lmp_alkyl.strucC,methane,50)

In [118]:
print lmp_alkyl.strucC.lat

66.550000 0.000000 0.000000
0.000000 66.550000 0.000000
0.000000 0.000000 66.550000


In [119]:
lmp_alkyl.strucC.tag += '_methane_x50'

In [120]:
lmp_alkyl.strucC.write_xyz()

In [121]:
for pk,p in lmp_alkyl.strucC.particles.iteritems():
    print p,p.mol,p.residue,p.resname

atom[0] C (C) 0 2 OCT
atom[1] H (H) 0 2 OCT
atom[2] H (H) 0 2 OCT
atom[3] H (H) 0 2 OCT
atom[4] C (C) 0 2 OCT
atom[5] H (H) 0 2 OCT
atom[6] H (H) 0 2 OCT
atom[7] C (C) 0 2 OCT
atom[8] H (H) 0 2 OCT
atom[9] H (H) 0 2 OCT
atom[10] C (C) 0 2 OCT
atom[11] H (H) 0 2 OCT
atom[12] H (H) 0 2 OCT
atom[13] C (C) 0 2 OCT
atom[14] H (H) 0 2 OCT
atom[15] C (C) 0 2 OCT
atom[16] H (H) 0 2 OCT
atom[17] H (H) 0 2 OCT
atom[18] C (C) 0 2 OCT
atom[19] H (H) 0 2 OCT
atom[20] H (H) 0 2 OCT
atom[21] C (C) 0 2 OCT
atom[22] H (H) 0 2 OCT
atom[23] H (H) 0 2 OCT
atom[24] H (H) 0 2 OCT
atom[25] C (C) 0 1 ETH
atom[26] H (H) 0 1 ETH
atom[27] H (H) 0 1 ETH
atom[28] C (C) 0 1 ETH
atom[29] H (H) 0 1 ETH
atom[30] H (H) 0 1 ETH
atom[31] H (H) 0 1 ETH
atom[32] C (C) 1 2 OCT
atom[33] H (H) 1 2 OCT
atom[34] H (H) 1 2 OCT
atom[35] H (H) 1 2 OCT
atom[36] C (C) 1 2 OCT
atom[37] H (H) 1 2 OCT
atom[38] H (H) 1 2 OCT
atom[39] C (C) 1 2 OCT
atom[40] H (H) 1 2 OCT
atom[41] H (H) 1 2 OCT
atom[42] C (C) 1 2 OCT
atom[43] H (H) 1 2 OC

In [122]:
lmp_alkyl.set_ffparam()

In [123]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_sp.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_alkyl.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [124]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_alkyl.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [125]:
os.chdir(lmp_alkyl.dir['scratch'])

In [126]:
lmp_alkyl.load_str('templates','in')
lmp_alkyl.load_str('templates','run')

In [127]:
lmp_alkyl.write_data()

In [128]:
lmp_alkyl.replacewrite_prop('in','input','in','%s.in'%(lmp_alkyl.tag))

In [129]:
lmp_alkyl.properties['input_in'] = lmp_alkyl.files['input']['in']
lmp_alkyl.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_alkyl.tag))

In [130]:
os.chdir(lmp_alkyl.dir['home'])
lmp_alkyl.dump_json()

In [131]:
lmp_alkyl.run()

In [132]:
os.chdir(lmp_alkyl.dir['scratch'])
lmp_alkyl.check()

In [133]:
pprint("Calculation:{} has status:{}".format(lmp_alkyl.tag,lmp_alkyl.meta['status']))

u'Calculation:lmp_alkyl has status:written'


In [134]:
lmp_alkyl.strucC.calc_center_mass()

In [146]:
groupset_i = streamm.Groups('mol',lmp_alkyl.strucC)
groupset_i.group_prop('mol','group_mol')

In [147]:
groupset_i.calc_cent_mass()
groupset_i.calc_radius_asphericity()
groupset_i.calc_dl()

In [148]:
groupset_i.write_cm_xyz()

In [149]:
import numpy as np

In [150]:
print np.mean(groupset_i.radius),groupset_i.strucC.unit_conf['length']

1.79932546227 ang


In [151]:
print groupset_i.strucC.lat.pbcs

[True, True, True]


In [152]:
groupset_i.group_nblist.radii_nblist(groupset_i.strucC.lat,groupset_i.cent_mass,groupset_i.radius,radii_buffer=5.25)

In [155]:
groupset_i.group_pbcs()

In [156]:
for gk_i,g_i in groupset_i.groups.iteritems():
    if( len(g_i.pkeys) == 32 ):
        print g_i.tag,groupset_i.group_nblist.calc_nnab(gk_i),g_i.mol 
        print g_i.cent_mass
        list_i = []
        for g_j in groupset_i.group_nblist.getnbs(gk_i):
            list_i += groupset_i.groups[g_j].pkeys
        groupset_i.strucC.shift_pos(-1.0*g_i.cent_mass)  # Place center of mass at origin
        groupset_i.strucC.write_xyz_list(list_i,xyz_file='{}_blob.xyz'.format(g_i.tag))
        groupset_i.strucC.shift_pos(g_i.cent_mass)  # Return center of mass 
        

group_mol_0 47 0
[ 38.326923   5.15297    0.311727]
group_mol_1 35 1
[ 36.599462  26.819205  37.318824]
group_mol_2 35 2
[  3.427126  43.098942  12.819743]
group_mol_3 34 3
[  2.385817  34.09571   44.931817]
group_mol_4 36 4
[ 12.99275   18.251691  29.674268]
group_mol_5 45 5
[ 17.256952   2.611683  24.930946]
group_mol_6 34 6
[ 33.107853  35.964402  24.485375]
group_mol_7 42 7
[ 18.156043  28.621305  46.606473]
group_mol_8 39 8
[ 29.279355  48.702058  37.262101]
group_mol_9 39 9
[ 46.718605  28.275142  16.173422]


Fancy aye! 